In [1]:
from data import *

folder = "/workspace/seq-vision/UCF101"
num_samples = 4       # 각 클래스별 사용할 영상 수 (예: 4)
num_frame = 50       # 각 영상에서 추출할 프레임 수
split_ratio = (6,2,2) # 예: 4개의 영상이면 train:2, valid:1, test:1 (비율은 상대적으로 적용됨)
batch_size = 4
num_workers = 0

# 1. 파일 읽어오기
files = load_avi_files(folder)
# 2. 파일명에서 label 추출 및 그룹화
class_to_files = group_files_by_label(folder, files)
# 2-2. label mapping 생성 (101개 클래스)
label2idx = create_label_mapping(class_to_files)
# 3. 각 클래스별 num_samples 영상 선택 및 num_frame 프레임 추출
class_to_videos = sample_videos_for_each_class(class_to_files, num_samples, num_frame)
# 4. 각 클래스별로 영상들을 split_ratio에 따라 train/valid/test로 분할
train_list, valid_list, test_list = split_videos(class_to_videos, ratio=split_ratio)
print(f"Train samples: {len(train_list)}, Valid samples: {len(valid_list)}, Test samples: {len(test_list)}")

Train samples: 202, Valid samples: 101, Test samples: 101


In [2]:
import os
import cv2

def save_videos(video_list, save_folder, label_mapping, fps=25):
    """
    주어진 video_list의 영상들을 save_folder에 mp4 파일로 저장합니다.
    파일 이름은 {class_index}_{원래 문자열 레이블}_{순번}.mp4 형식으로 저장됩니다.
    
    Args:
        video_list (list): [(label, frames), ...] 형식의 리스트.
                           frames는 영상의 프레임들을 담은 리스트 (각 프레임: numpy.ndarray).
        save_folder (str): 영상을 저장할 폴더 경로.
        label_mapping (dict): label string을 int index로 매핑한 딕셔너리.
        fps (int): 저장할 영상의 초당 프레임 수. (기본값 25)
    """
    os.makedirs(save_folder, exist_ok=True)
    video_counter = {}
    
    for label, frames in video_list:
        # label string을 mapping을 통해 정수 label로 변환
        class_idx = label_mapping[label]
        # 각 클래스별로 영상 순번 부여
        video_counter[class_idx] = video_counter.get(class_idx, 0) + 1
        filename = f"{class_idx}_{label}_{video_counter[class_idx]}.mp4"
        file_path = os.path.join(save_folder, filename)
        
        # 첫 프레임으로부터 영상 크기 확인 (height, width)
        height, width = frames[0].shape[:2]
        # mp4 저장을 위한 FourCC 코드 설정
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(file_path, fourcc, fps, (width, height))
        
        for frame in frames:
            out.write(frame)
        out.release()
        print(f"Saved video: {file_path}")

# 사용 예시 (이미 train_list, valid_list, test_list, label2idx가 생성되었다고 가정)
train_folder = "train"
valid_folder = "valid"
test_folder = "test"

save_videos(train_list, train_folder, label2idx)
save_videos(valid_list, valid_folder, label2idx)
save_videos(test_list, test_folder, label2idx)


Saved video: train/49_Knitting_1.mp4
Saved video: train/49_Knitting_2.mp4
Saved video: train/11_Billiards_1.mp4
Saved video: train/11_Billiards_2.mp4
Saved video: train/26_Drumming_1.mp4
Saved video: train/26_Drumming_2.mp4
Saved video: train/32_GolfSwing_1.mp4
Saved video: train/32_GolfSwing_2.mp4
Saved video: train/86_SumoWrestling_1.mp4
Saved video: train/86_SumoWrestling_2.mp4
Saved video: train/54_MoppingFloor_1.mp4
Saved video: train/54_MoppingFloor_2.mp4
Saved video: train/53_Mixing_1.mp4
Saved video: train/53_Mixing_2.mp4
Saved video: train/60_PlayingDhol_1.mp4
Saved video: train/60_PlayingDhol_2.mp4
Saved video: train/37_HandstandWalking_1.mp4
Saved video: train/37_HandstandWalking_2.mp4
Saved video: train/5_BandMarching_1.mp4
Saved video: train/5_BandMarching_2.mp4
Saved video: train/97_WalkingWithDog_1.mp4
Saved video: train/97_WalkingWithDog_2.mp4
Saved video: train/61_PlayingFlute_1.mp4
Saved video: train/61_PlayingFlute_2.mp4
Saved video: train/70_Punch_1.mp4
Saved video: